In [3]:
!pip install bayesian-optimization
# Bayesian Optimization used to find the maximum value of an unknown function while minimizing the number of iterations.

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score,confusion_matrix
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [5]:
df = pd.read_csv('/content/drive/MyDrive/ml-dataset/march-01/heart.csv')

In [6]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [7]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [9]:
df.head(50)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
5,58,0,0,100,248,0,0,122,0,1.0,1,0,2,1
6,58,1,0,114,318,0,2,140,0,4.4,0,3,1,0
7,55,1,0,160,289,0,0,145,1,0.8,1,1,3,0
8,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0
9,54,1,0,122,286,0,0,116,1,3.2,1,2,2,0


In [10]:
# Dataset Shuffle help to mixture the TARGET values
df1 = df.sample(frac=1).reset_index(drop=True)

In [11]:
df1.head(50)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,44,1,0,112,290,0,0,153,0,0.0,2,1,2,0
1,45,0,1,130,234,0,0,175,0,0.6,1,0,2,1
2,51,0,2,120,295,0,0,157,0,0.6,2,0,2,1
3,64,1,3,110,211,0,0,144,1,1.8,1,0,2,1
4,54,0,2,160,201,0,1,163,0,0.0,2,1,2,1
5,46,1,1,101,197,1,1,156,0,0.0,2,0,3,1
6,45,0,0,138,236,0,0,152,1,0.2,1,0,2,1
7,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0
8,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
9,70,1,0,130,322,0,0,109,0,2.4,1,3,2,0


In [12]:
df1.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [13]:
X = df1.drop(columns=['target'])
y = df1['target']

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler1 = MinMaxScaler()

In [15]:
# Scalling much needed for Distance Based Matrix -- ( KNN , LR )
MinMax_scaled_X = scaler1.fit_transform(X)
MinMax_scaled_X

array([[0.3125    , 1.        , 0.        , ..., 1.        , 0.25      ,
        0.66666667],
       [0.33333333, 0.        , 0.33333333, ..., 0.5       , 0.        ,
        0.66666667],
       [0.45833333, 0.        , 0.66666667, ..., 1.        , 0.        ,
        0.66666667],
       ...,
       [0.35416667, 1.        , 0.33333333, ..., 1.        , 0.        ,
        1.        ],
       [0.77083333, 0.        , 0.        , ..., 0.5       , 0.5       ,
        1.        ],
       [0.64583333, 0.        , 0.66666667, ..., 1.        , 0.25      ,
        0.66666667]])

# Random Serach

In [16]:
# split a dataset into train and test sets
from sklearn.model_selection import train_test_split as tts

# global X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = tts(MinMax_scaled_X, y, test_size=0.2,random_state=0)


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


<h1>Random Search</h1>
<details>
  <summary> <font color="blue">Click</summary>
  <font color="black">
  - Quick <br>
  - Can miss a combination
</details>


In [18]:
rf_param_grid={
    'max_depth':list(range(8,100,10)),
    'n_estimators':list(range(50,2000,100)),
    'min_samples_split':[2,6,8],
    'min_samples_leaf':[2,6,8],
    'bootstrap':[True,False]
}

In [19]:
rf_param_grid

{'max_depth': [8, 18, 28, 38, 48, 58, 68, 78, 88, 98],
 'n_estimators': [50,
  150,
  250,
  350,
  450,
  550,
  650,
  750,
  850,
  950,
  1050,
  1150,
  1250,
  1350,
  1450,
  1550,
  1650,
  1750,
  1850,
  1950],
 'min_samples_split': [2, 6, 8],
 'min_samples_leaf': [2, 6, 8],
 'bootstrap': [True, False]}

In [20]:
m = RandomForestClassifier(n_jobs=-1)

In [21]:
# m_r = GridSearchCV(param_grid=rf_param_grid,estimator=m,scoring='accuracy',cv=4)

In [22]:
m_r = RandomizedSearchCV(param_distributions=rf_param_grid,estimator=m,scoring='accuracy',cv=4,n_jobs=-1,n_iter=10)

In [23]:
%time m_r.fit(X_train,y_train)

CPU times: user 3.42 s, sys: 459 ms, total: 3.88 s
Wall time: 1min 35s


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [8, 18, 28, 38, 48, 58, 68,
                                                      78, 88, 98],
                                        'min_samples_leaf': [2, 6, 8],
                                        'min_samples_split': [2, 6, 8],
                                        'n_estimators': [50, 150, 250, 350, 450,
                                                         550, 650, 750, 850,
                                                         950, 1050, 1150, 1250,
                                                         1350, 1450, 1550, 1650,
                                                         1750, 1850, 1950]},
                   scoring='accuracy')

In [24]:
m_r.best_params_,m_r.best_index_

({'n_estimators': 950,
  'min_samples_split': 6,
  'min_samples_leaf': 2,
  'max_depth': 48,
  'bootstrap': False},
 3)

In [25]:
for param , score in zip(m_r.cv_results_['params'],m_r.cv_results_['mean_test_score']):
  print(param,score)

{'n_estimators': 750, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_depth': 28, 'bootstrap': True} 0.9109756097560976
{'n_estimators': 50, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_depth': 68, 'bootstrap': False} 0.974390243902439
{'n_estimators': 550, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_depth': 38, 'bootstrap': True} 0.9536585365853658
{'n_estimators': 950, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_depth': 48, 'bootstrap': False} 0.9817073170731707
{'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_depth': 98, 'bootstrap': True} 0.9170731707317074
{'n_estimators': 1950, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_depth': 88, 'bootstrap': True} 0.9073170731707317
{'n_estimators': 1750, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_depth': 38, 'bootstrap': True} 0.8890243902439025
{'n_estimators': 1750, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_depth': 38, 'bootstrap': True} 0.9121951219512195
{'n_es

In [26]:
rf_bp = m_r.best_params_

In [27]:
rf_bp

{'n_estimators': 950,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_depth': 48,
 'bootstrap': False}

## Hyper parameter tuning .
#


In [28]:
rf_classifier = RandomForestClassifier(n_estimators=rf_bp["n_estimators"],
                               min_samples_split=rf_bp["min_samples_split"],
                               min_samples_leaf=rf_bp["min_samples_leaf"],
                               max_depth=rf_bp["max_depth"],
                               bootstrap = rf_bp['bootstrap']
                               )

In [29]:
rf_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, max_depth=48, min_samples_leaf=2,
                       min_samples_split=6, n_estimators=950)

In [30]:
y_pred_randomCV = rf_classifier.predict(X_test)

In [31]:
y_pred_randomCV

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1])

In [32]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

y_pred_randomCV = rf_classifier.predict(X_test)

print(confusion_matrix(y_test,y_pred_randomCV))
print(accuracy_score(y_test,y_pred_randomCV))
print(classification_report(y_test,y_pred_randomCV))
acc = accuracy_score(y_test,y_pred_randomCV)

[[ 85   0]
 [  6 114]]
0.9707317073170731
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        85
           1       1.00      0.95      0.97       120

    accuracy                           0.97       205
   macro avg       0.97      0.97      0.97       205
weighted avg       0.97      0.97      0.97       205



In [33]:
acc

0.9707317073170731

In [34]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

print("accuracy: ",accuracy_score(y_test,y_pred_randomCV))
print("precision score : ",precision_score(y_test,y_pred_randomCV))
print("recall_score: ",recall_score(y_test,y_pred_randomCV))
print("f1 score : ",f1_score(y_test,y_pred_randomCV))
print("noc_auc_score: ",roc_auc_score(y_test,y_pred_randomCV))

accuracy:  0.9707317073170731
precision score :  1.0
recall_score:  0.95
f1 score :  0.9743589743589743
noc_auc_score:  0.975


# Baysian Optimization
- Objective function : loss function to minimize
- Domain Space : range of input values
- Optimization algorithm : Search algorithm Baysian Optimization builds a probability model of the objective function and uses of it to select hyperprameter to evalute in the true objective function

In [35]:
!pip install hyperopt

In [36]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
# Trials -> object to minimize the loss functions
#fmin - loss function -> tree based structure

In [52]:
space = {'criterion': hp.choice('criterion',['gini','entropy']),
         'n_estimators':hp.choice('n_estimators',[10,50,100,200]),
         'min_samples_leaf':hp.choice('min_samples_leaf',[1,2,3]),
         'min_samples_split':hp.choice('min_samples_split',[2,3,4]),
         'max_depth':hp.choice('max_depth',[3,5,10]),
         'max_features':hp.choice('max_features',['sqrt','log2',None]),
         'max_leaf_nodes':hp.choice('max_leaf_nodes',[2,3,4,None])
}

In [49]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7adb7262b190>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7adb63cff6a0>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7adb62f7ba30>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7adb62f7bbe0>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7adb62f7b2b0>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7adb62f78c40>,
 'max_leaf_nodes': <hyperopt.pyll.base.Apply at 0x7adb62f78880>}

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(space):

  RF_classifier = RandomForestClassifier(criterion=space['criterion'],
      n_estimators=space['n_estimators'],
      min_samples_split=space['min_samples_split'],
      min_samples_leaf=space['min_samples_leaf'],
      max_depth=space['max_depth'],
      max_features=space['max_features'],
      max_leaf_nodes=space['max_leaf_nodes']
      )


  accuracy = cross_val_score(RF_classifier,X_train,y_train,cv=5).mean()

  return {'loss': -accuracy , 'status':STATUS_OK}


In [ ]:
trials = Trials()

best = fmin(fn = objective,
            space=space,
            algo=tpe.suggest,
            max_evals=80,
            trials=trials)

best

 14%|█▍        | 11/80 [00:13<01:06,  1.04trial/s, best loss: -0.974390243902439]

### 2:23:30 min